## Packages

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, glob

from pytrends.request import TrendReq
from datetime import datetime

## Yfinance

In [3]:
#FTSE 100 tickers list
tickers = ["BP.L", "PSON.L", "BT-A.L", 
           "BDEV.L", "AZN.L", "BARC.L", 
           "LAND.L", "LGEN.L", "BA.L",
          "JET.L"]

In [4]:
#Data Collection from yfinance
for stock in tickers:
    stock_data=yf.Ticker(stock).history(start='2017-01-01', end = '2021-06-30')
    stock_data.to_csv("/Users/josephbrennan/Desktop/Data/StockData/stock_data_{}.csv".format(stock))

## Merge Google Trend csvs

In [5]:
#Select company:
company = 'lgen'
#Select ticker:
tick = 'LGEN.L'

In [6]:
#Company keywords list to 5
bp = ['British Petroleum','BP', 'oil', 'gas', 'petroleum', 'spill']
pson = ['Pearson', 'education', 'Edexcel', 'penguin', 'books', 'publisher']
bt = ['BT', 'EE', 'data', 'communications', 'network', 'services']
bdev = ['Barratt Developments', 'homes', 'site', 'property', 'building', 'developments']
azn = ['AstraZeneca', 'drug', 'treatment', 'research', 'MedImmune', 'Covid']
barc = ['Barclays', 'bank', 'capital', 'business', 'London', 'banking']
land = ['Land Securities Group', 'WPG', 'property', 'securities', 'portfolio', 'investment']
lgen = ['Legal & General', 'insurance', 'investment', 'pensions', 'assurance', 'business']
bae = ['BAE Systems', 'defence', 'aircraft', 'aerospace', 'Airbus', 'government']
jet = ['Just Eat', 'food', 'restaurants', 'takeaway', 'delivery', 'SkipTheDishes']

In [7]:
#Save Keyword CSVs
for kw in lgen:
    df = pd.read_csv(f'/Users/josephbrennan/Desktop/Data/Keywords/{company}/{kw}.csv')
    i = [2,3,4]
    df.drop(df.columns[i], axis=1, inplace=True)
    df.rename(columns={'date':'Date',kw+'_unscaled': kw}, inplace=True)
    df['Date'] = pd.to_datetime(df.Date)
    df.index = pd.DatetimeIndex(df.Date)
    df.drop('Date', axis=1, inplace=True)
    df.to_csv(f'/Users/josephbrennan/Desktop/Data/Keywords/{company}/' + ''.join(kw) + '.csv')

In [8]:
#File Path
path = '/Users/josephbrennan/Desktop/Data/Keywords/{}'.format(company)

#Change variable to company you need data from e.g. bp to jet
all_files = glob.glob(os.path.join(path, '*.csv'))

all_df = []
for f in all_files:
    df = pd.read_csv(f, sep = ',')
    #df['file'] = f.split('/')[-1]
    all_df.append(df)

cols = ['Date']

df_merged = pd.concat([x.set_index(cols) for x in all_df], axis=1)
df_merged

,Legal & General,assurance,insurance,business,pensions,investment
Date,,,,,,
2017-01-01,0,36,42,47,42,46
2017-01-02,63,73,66,62,67,63
2017-01-03,30,87,95,83,87,77
2017-01-04,60,85,99,89,95,83
2017-01-05,15,85,96,90,75,89
...,...,...,...,...,...,...
2021-06-26,19,61,63,65,46,71
2021-06-27,0,55,56,64,39,65
2021-06-28,17,97,91,93,73,87


In [10]:
#Save new merged keyword frame
df_merged.to_csv('/Users/josephbrennan/Desktop/Data/KeysMerged/{}_merged.csv'.format(company))

## Merge CSVs

In [11]:
keyword_df = pd.read_csv('/Users/josephbrennan/Desktop/Data/KeysMerged/{}_merged.csv'.format(company))
stock_df = pd.read_csv('/Users/josephbrennan/Desktop/Data/StockData/stock_data_{}.csv'.format(tick))

merged_data = pd.merge(stock_df, keyword_df, how='left', on = 'Date')

#Drop Dividends and Stock Splits
merged_data.drop(['Dividends', 'Stock Splits'], axis = 1, inplace = True)
merged_data

,Date,Open,High,Low,Close,Volume,Legal & General,assurance,insurance,business,pensions,investment
0,2017-01-03,179.480652,180.994348,179.048164,179.480652,15077668,30,87,95,83,87,77
1,2017-01-04,179.696929,181.787262,179.336526,181.787262,14391239,60,85,99,89,95,83
2,2017-01-05,181.354802,181.720968,177.390369,178.327408,14486260,15,85,96,90,75,89
3,2017-01-06,177.750760,179.949209,177.246187,179.913177,13498874,79,83,85,78,71,82
4,2017-01-09,179.913149,180.489785,176.885756,179.336502,11568808,59,89,90,88,74,87
...,...,...,...,...,...,...,...,...,...,...,...,...
1130,2021-06-23,263.723740,264.508913,261.270040,261.368164,11306170,100,88,90,95,70,92
1131,2021-06-24,261.760762,262.155295,257.834842,258.423706,19962490,52,83,87,94,97,88
1132,2021-06-25,259.405191,260.877411,255.970010,260.877411,13012606,18,85,84,85,60,85
1133,2021-06-28,259.994101,260.681149,255.675594,255.871902,8308010,17,97,91,93,73,87


In [12]:
keyword_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642 entries, 0 to 1641
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date             1642 non-null   object
 1   Legal & General  1642 non-null   int64 
 2   assurance        1642 non-null   int64 
 3   insurance        1642 non-null   int64 
 4   business         1642 non-null   int64 
 5   pensions         1642 non-null   int64 
 6   investment       1642 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 89.9+ KB


In [13]:
stock_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1135 entries, 0 to 1134
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          1135 non-null   object 
 1   Open          1135 non-null   float64
 2   High          1135 non-null   float64
 3   Low           1135 non-null   float64
 4   Close         1135 non-null   float64
 5   Volume        1135 non-null   int64  
 6   Dividends     1135 non-null   float64
 7   Stock Splits  1135 non-null   int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 71.1+ KB


In [14]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1135 entries, 0 to 1134
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             1135 non-null   object 
 1   Open             1135 non-null   float64
 2   High             1135 non-null   float64
 3   Low              1135 non-null   float64
 4   Close            1135 non-null   float64
 5   Volume           1135 non-null   int64  
 6   Legal & General  1135 non-null   int64  
 7   assurance        1135 non-null   int64  
 8   insurance        1135 non-null   int64  
 9   business         1135 non-null   int64  
 10  pensions         1135 non-null   int64  
 11  investment       1135 non-null   int64  
dtypes: float64(4), int64(7), object(1)
memory usage: 115.3+ KB


In [15]:
merged_data.isna().sum()

Date               0
Open               0
High               0
Low                0
Close              0
Volume             0
Legal & General    0
assurance          0
insurance          0
business           0
pensions           0
investment         0
dtype: int64

In [16]:
merged_data.fillna(value = 0, axis=1, inplace=True)

In [17]:
merged_data.isna().sum()

Date               0
Open               0
High               0
Low                0
Close              0
Volume             0
Legal & General    0
assurance          0
insurance          0
business           0
pensions           0
investment         0
dtype: int64

In [18]:
#Save Company Merged CSV for Later Use
merged_data.to_csv('/Users/josephbrennan/Desktop/Data/Merged/{}_merged.csv'.format(company))